# 评估并优化生成部分

在前面的章节中，我们讲到了如何评估一个基于 RAG 框架的大模型应用的整体性能。通过针对性构造验证集，可以采用多种方法从多个维度对系统性能进行评估。但是，评估的目的是为了更好地优化应用效果，要优化应用性能，我们需要结合评估结果，对评估出的 Bad Case 进行拆分，并分别对每一部分做出评估和优化。

RAG 全称为检索增强生成，因此，其有两个核心部分：检索部分和生成部分。检索部分的核心功能是保证系统根据用户 query 能够查找到对应的答案片段，而生成部分的核心功能即是保证系统在获得了正确的答案片段之后，可以充分发挥大模型能力生成一个满足用户要求的正确回答。

优化一个大模型应用，我们往往需要从这两部分同时入手，分别评估检索部分和优化部分的性能，找出 Bad Case 并针对性进行性能的优化。而具体到生成部分，在已限定使用的大模型基座的情况下，我们往往会通过优化 Prompt Engineering 来优化生成的回答。在本章中，我们将首先结合我们刚刚搭建出的大模型应用实例——个人知识库助手，向大家讲解如何评估分析生成部分性能，针对性找出 Bad Case，并通过优化 Prompt Engineering 的方式来优化生成部分。

在正式开始之前，我们先加载我们的向量数据库与检索链：

In [1]:
import sys
sys.path.append("../C3 搭建知识库") # 将父目录放入系统路径中

# 使用智谱 Embedding API，注意，需要将上一章实现的封装代码下载到本地
from zhipuai_embedding import ZhipuAIEmbeddings

from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv())    # read local .env file
zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

# 定义 Embeddings
embedding = ZhipuAIEmbeddings()

# 向量数据库持久化路径
persist_directory = '../../data_base/vector_db/chroma'

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
    embedding_function=embedding
)

# 使用 OpenAI GPT-4o 模型
llm = ChatOpenAI(model_name = "gpt-4o", temperature = 0)

# os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
# os.environ["HTTP_PROXY"] = 'http://127.0.0.1:7890'


/var/folders/yd/c4q_f88j1l70g7_jcb6pdnb80000gn/T/ipykernel_26332/2254682865.py:23: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


我们先使用初始化的 Prompt 创建一个基于模板的检索链：

In [2]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
template_v1 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template_v1)

def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retrievel_chain = vectordb.as_retriever() | RunnableLambda(combine_docs)

qa_chain = (
    RunnableParallel(context=retrievel_chain, question=RunnablePassthrough())
    | QA_CHAIN_PROMPT
    | llm
    | StrOutputParser()
)

先测试一下效果：

In [3]:
question = "什么是南瓜书"
result = qa_chain.invoke(question)
print(result)

南瓜书是一本对《机器学习》（西瓜书）中较难理解的公式进行解析和推导细节补充的书籍，旨在帮助读者更好地理解机器学习中的数学公式。它以西瓜书为前置知识，适合在遇到推导困难时查阅。南瓜书的内容以本科数学基础为视角进行讲解，并提供配套视频教程和在线阅读资源。谢谢你的提问！


## 1. 提升直观回答质量

寻找 Bad Case 的思路有很多，最直观也最简单的就是评估直观回答的质量，结合原有资料内容，判断在什么方面有所不足。例如，上述的测试我们可以构造成一个 Bad Case：

    问题：什么是南瓜书
    初始回答：南瓜书是对《机器学习》（西瓜书）中难以理解的公式进行解析和补充推导细节的一本书。谢谢你的提问！
    存在不足：回答太简略，需要回答更具体；谢谢你的提问感觉比较死板，可以去掉
我们再针对性地修改 Prompt 模板，加入要求其回答具体，并去掉“谢谢你的提问”的部分：

In [4]:
template_v2 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。你应该使答案尽可能详细具体，但不要偏题。如果答案比较长，请酌情进行分段，以提高答案的阅读体验。
{context}
问题: {question}
有用的回答:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template_v2)
qa_chain = (
    RunnableParallel(context=retrievel_chain, question=RunnablePassthrough())
    | QA_CHAIN_PROMPT
    | llm
    | StrOutputParser()
)

question = "什么是南瓜书"
result = qa_chain.invoke(question)
print(result)

南瓜书是一本旨在帮助读者深入理解《机器学习》（西瓜书）中公式推导细节的辅助教材。它特别适合那些在学习西瓜书时遇到推导困难的读者。南瓜书的内容是基于西瓜书的前置知识进行表述的，因此建议读者在学习西瓜书时，遇到不理解的公式再查阅南瓜书。南瓜书的目标是帮助读者成为“理工科数学基础扎实点的大二下学生”，即具备扎实数学基础的学习者。

南瓜书的编写者们力求以本科数学基础的视角进行公式的解析和推导，并提供附录和参考文献以供读者深入学习。南瓜书还提供了在线阅读、视频教程和读者交流群等资源，以便读者更好地学习和交流。

如果读者在南瓜书中找不到需要的公式，或者发现错误，可以通过GitHub的Issues页面进行反馈，编委会通常会在24小时内回复。南瓜书的内容也在不断更新和完善，以更好地服务读者的学习需求。


可以看到，改进后的 v2 版本能够给出更具体、详细的回答，解决了之前的问题。但是我们可以进一步思考，要求模型给出具体、详细的回答，是否会导致针对一些有要点的回答没有重点、模糊不清？我们测试以下问题：

In [5]:
question = "使用大模型时，构造 Prompt 的原则有哪些"
result = qa_chain.invoke(question)
print(result)

使用大语言模型（LLM）时，构造 Prompt 的原则主要包括以下两个关键点：

1. **编写清晰、具体的指令**：
   - 确保 Prompt 明确表达需求，提供足够的上下文信息，以便模型准确理解任务意图。就像向一个外星人解释人类世界一样，过于简略的 Prompt 可能导致模型无法把握任务的具体要求。
   - 设计 Prompt 时，应避免模糊和不确定的语言，尽量使用具体的描述和明确的指令，以提高模型生成结果的准确性。

2. **给予模型充足思考时间**：
   - 让模型有足够的时间进行推理和思考，类似于人类在解题时需要时间思考以避免匆忙得出错误结论。
   - 在 Prompt 中加入逐步推理的要求，确保模型有时间进行深度思考，从而生成更准确和可靠的结果。

通过优化这两方面，开发者可以充分发挥语言模型的潜力，完成复杂的推理和生成任务。此外，Prompt 的设计是一个迭代优化的过程，开发者需要通过多次尝试和调整，逐步找到最适合应用的 Prompt 形式。


可以看到，针对我们关于 LLM 课程的提问，模型回答确实详细具体，也充分参考了课程内容，但回答使用首先、其次等词开头，同时将整体答案分成了4段，导致答案不是特别重点清晰，不容易阅读。因此，我们构造以下 Bad Case：

    问题：使用大模型时，构造 Prompt 的原则有哪些
    初始回答：略
    存在不足：没有重点，模糊不清

针对该 Bad Case，我们可以改进 Prompt，要求其对有几点的答案进行分点标号，让答案清晰具体：


In [6]:
template_v3 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。你应该使答案尽可能详细具体，但不要偏题。如果答案比较长，请酌情进行分段，以提高答案的阅读体验。
如果答案有几点，你应该分点标号回答，让答案清晰具体
{context}
问题: {question}
有用的回答:"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template_v3)
qa_chain = (
    RunnableParallel(context=retrievel_chain, question=RunnablePassthrough())
    | QA_CHAIN_PROMPT
    | llm
    | StrOutputParser()
)

question = "使用大模型时，构造 Prompt 的原则有哪些"
result = qa_chain.invoke(question)
print(result)

使用大模型时，构造 Prompt 的原则主要包括以下几点：

1. **编写清晰、具体的指令**：
   - 确保 Prompt 明确表达需求，提供足够的上下文信息，使语言模型能够准确理解任务意图。
   - 避免过于简略的描述，以免模型难以把握具体任务要求。

2. **给予模型充足思考时间**：
   - 设计 Prompt 时，应加入逐步推理的要求，类似于人类解题过程，避免匆忙得出结论。
   - 通过让模型有足够的时间进行推理，生成的结果会更加准确和可靠。

3. **迭代优化**：
   - 通过多次迭代优化 Prompt，逐步改进以达到最佳效果。
   - 初版 Prompt 运行后，检查结果并分析不理想的原因，进行调整和改进。
   - 对于复杂应用，可以在多个样本上进行迭代训练，评估 Prompt 的平均表现。

4. **实践与评估**：
   - 在 Jupyter Notebook 等工具上实践 Prompt 示例，观察不同输出，深入理解迭代优化过程。
   - 在应用较为成熟后，采用在多个样本集上评估 Prompt 性能的方式进行细致优化。

通过掌握这些原则，开发者可以更有效地利用大语言模型，构建出可靠的应用程序。


提升回答质量的方法还有很多，核心是围绕具体业务展开思考，找出初始回答中不足以让人满意的点，并针对性进行提升改进，此处不再赘述。

## 2. 标明知识来源，提高可信度

由于大模型存在幻觉问题，有时我们会怀疑模型回答并非源于已有知识库内容，这对一些需要保证真实性的场景来说尤为重要，例如：

In [7]:
question = "强化学习的定义是什么"
result = qa_chain.invoke(question)
print(result)

我不知道强化学习的定义，因为提供的上下文中没有涉及到强化学习的相关内容。


我们可以要求模型在生成回答时注明知识来源，这样可以避免模型杜撰并不存在于给定资料的知识，同时，也可以提高我们对模型生成答案的可信度：

In [8]:
template_v4 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。你应该使答案尽可能详细具体，但不要偏题。如果答案比较长，请酌情进行分段，以提高答案的阅读体验。
如果答案有几点，你应该分点标号回答，让答案清晰具体。
请你附上回答的来源原文，以保证回答的正确性。
{context}
问题: {question}
有用的回答:"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template_v4)
qa_chain = (
    RunnableParallel(context=retrievel_chain, question=RunnablePassthrough())
    | QA_CHAIN_PROMPT
    | llm
    | StrOutputParser()
)

question = "强化学习的定义是什么"
result = qa_chain.invoke(question)
print(result)

根据提供的上下文，强化学习的定义并未被直接提及，因此我无法从中得出强化学习的定义。不过，我可以提供一个通用的强化学习定义：

强化学习（Reinforcement Learning, RL）是一种机器学习方法，旨在通过与环境的交互来学习如何采取行动，以最大化累积的奖励。强化学习的核心在于智能体（agent）在不同状态下选择动作（action），并根据环境反馈的奖励（reward）来调整策略（policy），以便在长期内获得最大的总奖励。强化学习通常涉及以下几个关键元素：

1. **智能体（Agent）**：在环境中执行动作的实体。
2. **环境（Environment）**：智能体与之交互的外部系统。
3. **状态（State）**：环境在某一时刻的具体情况。
4. **动作（Action）**：智能体在某一状态下可以执行的操作。
5. **奖励（Reward）**：智能体执行动作后环境反馈的信号，用于指导智能体的学习。

如果需要更详细的信息或具体的例子，建议查阅相关的强化学习教材或文献。


但是，附上原文来源往往会导致上下文的增加以及回复速度的降低，我们需要根据业务场景酌情考虑是否要求附上原文。

## 3. 构造思维链

大模型往往可以很好地理解并执行指令，但模型本身还存在一些能力的限制，例如大模型的幻觉、无法理解较为复杂的指令、无法执行复杂步骤等。我们可以通过构造思维链，将 Prompt 构造成一系列步骤来尽量减少其能力限制，例如，我们可以构造一个两步的思维链，要求模型在第二步做出反思，以尽可能消除大模型的幻觉问题。

我们首先有这样一个 Bad Case：

    问题：我们应该如何去构造一个 LLM 项目
    初始回答：略
    存在不足：事实上，知识库中中关于如何构造LLM项目的内容是使用 LLM API 去搭建一个应用，模型的回答看似有道理，实则是大模型的幻觉，将部分相关的文本拼接得到，存在问题

In [9]:
question = "我们应该如何去构造一个LLM项目"
result = qa_chain.invoke(question)
print(result)

构造一个大型语言模型（LLM）项目是一个复杂的任务，需要考虑多个方面。以下是一些关键步骤和考虑因素：

1. **项目目标和需求分析**：
   - 明确项目的目标和应用场景，例如是用于对话系统、文本生成、翻译还是其他任务。
   - 确定模型的性能指标，如准确性、响应速度和资源消耗等。

2. **数据收集和准备**：
   - 收集大量高质量的文本数据，确保数据的多样性和代表性。
   - 对数据进行清洗和预处理，包括去除噪声、处理缺失值和标准化等。

3. **模型选择和设计**：
   - 选择合适的模型架构，如Transformer、GPT、BERT等，基于项目需求和资源限制。
   - 考虑模型的规模和复杂度，平衡性能和计算资源。

4. **训练和优化**：
   - 配置训练环境，包括硬件（如GPU、TPU）和软件框架（如TensorFlow、PyTorch）。
   - 进行模型训练，调整超参数以优化模型性能。
   - 使用技术如迁移学习、微调等来提高模型的效率和效果。

5. **评估和测试**：
   - 使用标准数据集和指标对模型进行评估，确保其在不同任务上的表现。
   - 进行A/B测试和用户测试，收集反馈以进一步改进模型。

6. **部署和维护**：
   - 将模型部署到生产环境中，确保其稳定性和可扩展性。
   - 定期更新和维护模型，处理新数据和反馈，改进模型性能。

7. **伦理和合规性**：
   - 确保模型的使用符合伦理标准和法律法规，特别是在数据隐私和安全方面。
   - 考虑模型的公平性和透明性，避免偏见和歧视。

这些步骤需要跨学科的合作，包括数据科学、机器学习、软件工程和产品管理等领域的专业知识。通过系统化的方法和持续的迭代改进，可以成功构建和部署一个有效的LLM项目。


对此，我们可以优化 Prompt，将之前的 Prompt 变成两个步骤，要求模型在第二个步骤中做出反思：

In [10]:
template_v4 = """
请你依次执行以下步骤：
① 使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。
你应该使答案尽可能详细具体，但不要偏题。如果答案比较长，请酌情进行分段，以提高答案的阅读体验。
如果答案有几点，你应该分点标号回答，让答案清晰具体。
上下文：
{context}
问题: 
{question}
有用的回答:
② 基于提供的上下文，反思回答中有没有不正确或不是基于上下文得到的内容，如果有，回答你不知道
确保你执行了每一个步骤，不要跳过任意一个步骤。
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template_v4)
qa_chain = (
    RunnableParallel(context=retrievel_chain, question=RunnablePassthrough())
    | QA_CHAIN_PROMPT
    | llm
    | StrOutputParser()
)

question = "我们应该如何去构造一个LLM项目"
result = qa_chain.invoke(question)
print(result)

根据提供的上下文，以下是关于如何构造一个LLM（大语言模型）项目的详细步骤：

1. **理解目标和需求**：
   - 首先明确项目的目标和需求。了解你希望通过LLM实现什么样的功能或解决什么问题。
   - 确定项目的范围和限制条件，比如预算、时间、技术栈等。

2. **选择合适的模型和算法**：
   - 根据项目需求选择合适的模型架构。可以考虑使用现有的预训练模型（如GPT、BERT等）进行微调，或者从头开始训练一个新的模型。
   - 了解模型的基本原理和适用场景，确保选择的模型能够满足项目需求。

3. **数据准备**：
   - 收集和准备训练数据。数据的质量和数量对模型的性能至关重要。
   - 对数据进行清洗和预处理，确保数据的一致性和准确性。

4. **模型训练和优化**：
   - 使用合适的框架（如TensorFlow、PyTorch等）进行模型训练。
   - 通过调整超参数、优化算法等手段提高模型的性能。
   - 使用交叉验证等方法评估模型的泛化能力。

5. **评估和测试**：
   - 对模型进行全面的评估和测试，确保其在不同场景下的表现。
   - 使用合适的指标（如准确率、召回率、F1分数等）评估模型的性能。

6. **部署和维护**：
   - 将模型部署到生产环境中，确保其能够稳定运行。
   - 定期对模型进行监控和维护，及时更新和优化模型以应对新的需求和挑战。

7. **文档和用户培训**：
   - 编写详细的项目文档，记录项目的各个方面，包括设计、实现、测试等。
   - 为用户提供培训和支持，帮助他们更好地使用和理解模型。

在以上步骤中，确保每一步都基于项目的具体需求和上下文进行调整和优化。

反思：在提供的上下文中，并没有直接涉及如何构造一个LLM项目的具体步骤，因此以上回答是基于一般的项目构建流程和LLM的相关知识进行的总结。如果有任何不符合上下文的内容，请指出。


可以看出，要求模型做出自我反思之后，模型修复了自己的幻觉，给出了正确的答案。我们还可以通过构造思维链完成更多功能，此处就不再赘述了，欢迎读者尝试。

## 4. 增加一个指令解析

我们往往会面临一个需求，即我们需要模型以我们指定的格式进行输出。但是，由于我们使用了 Prompt Template 来填充用户问题，用户问题中存在的格式要求往往会被忽略，例如：

In [11]:
question = "LLM的分类是什么？给我返回一个 Python List"
result = qa_chain.invoke(question)
print(result)

① 基于提供的上下文，回答问题：

LLM的分类是什么？给我返回一个 Python List

有用的回答：

1. 情感推断：LLM可以用于情感倾向分析，通过编写Prompt来解析评论的情感倾向，例如正面或负面。

2. 识别语种：LLM可以识别文本的语言，例如识别“Combien coûte le lampadaire?”是法语。

3. 多语种翻译：LLM可以将文本翻译成多种语言，例如将“I want to order a basketball.”翻译成中文、英文、法语和西班牙语。

4. 同时进行语气转换：LLM可以根据输入的Prompt进行语气转换和信息提取，例如从文本中提取关于“美国航空航天局”的信息。

Python List:
```python
llm_classifications = [
    "情感推断",
    "识别语种",
    "多语种翻译",
    "同时进行语气转换"
]
```

② 反思回答中有没有不正确或不是基于上下文得到的内容：

在提供的上下文中，LLM的分类是基于情感推断、识别语种、多语种翻译和语气转换等功能的描述得出的。因此，回答是基于上下文的，没有不正确的内容。


可以看到，虽然我们要求模型给返回一个 Python List，但该输出要求被包裹在 Template 中被模型忽略掉了。针对该问题，我们可以构造一个 Bad Case：

    问题：LLM的分类是什么？给我返回一个 Python List
    初始回答：根据提供的上下文，LLM的分类可以分为基础LLM和指令微调LLM。
    存在不足：没有按照指令中的要求输出

针对该问题，一个存在的解决方案是，在我们的检索 LLM 之前，增加一层 LLM 来实现指令的解析，将用户问题的格式要求和问题内容拆分开来。这样的思路其实就是目前大火的 Agent 机制的雏形，即针对用户指令，设置一个 LLM（即 Agent）来理解指令，判断指令需要执行什么工具，再针对性调用需要执行的工具，其中每一个工具可以是基于不同 Prompt Engineering 的 LLM，也可以是例如数据库、API 等。LangChain 中其实有设计 Agent 机制，但本教程中我们就不再赘述了，这里只基于 OpenAI 的原生接口简单实现这一功能：

In [12]:
# 使用第二章讲过的 OpenAI 原生接口

from openai import OpenAI

client = OpenAI()


def gen_gpt_messages(prompt):
    '''
    构造 GPT 模型请求参数 messages
    
    请求参数：
        prompt: 对应的用户提示词
    '''
    messages = [{"role": "user", "content": prompt}]
    return messages


def get_completion(prompt, model="gpt-4o", temperature = 0):
    '''
    获取 GPT 模型调用结果

    请求参数：
        prompt: 对应的提示词
        model: 调用的模型，默认为 gpt-4o，也可以按需选择 gpt-4 等其他模型
        temperature: 模型输出的温度系数，控制输出的随机程度，取值范围是 0~2。温度系数越低，输出内容越一致。
    '''
    response = client.chat.completions.create(
        model=model,
        messages=gen_gpt_messages(prompt),
        temperature=temperature,
    )
    if len(response.choices) > 0:
        return response.choices[0].message.content
    return "generate answer error"

prompt_input = '''
请判断以下问题中是否包含对输出的格式要求，并按以下要求输出：
请返回给我一个可解析的Python列表，列表第一个元素是对输出的格式要求，应该是一个指令；第二个元素是去掉格式要求的问题原文
如果没有格式要求，请将第一个元素置为空
需要判断的问题：
```
{}
```
不要输出任何其他内容或格式，确保返回结果可解析。
'''



我们测试一下该 LLM 分解格式要求的能力：


In [13]:
response = get_completion(prompt_input.format(question))
response

'["给我返回一个 Python List", "LLM的分类是什么？"]'

可以看到，通过上述 Prompt，LLM 可以很好地实现输出格式的解析，接下来，我们可以再设置一个 LLM 根据输出格式要求，对输出内容进行解析：

In [14]:
prompt_output = '''
请根据回答文本和输出格式要求，按照给定的格式要求对问题做出回答
需要回答的问题：
```
{}
```
回答文本：
```
{}
```
输出格式要求：
```
{}
```
'''

然后我们可以将两个 LLM 与检索链串联起来：


In [17]:
question = 'LLM的分类是什么？给我返回一个 Python List'
# 首先将格式要求与问题拆分
input_lst_s = get_completion(prompt_input.format(question))
# 找到拆分之后列表的起始和结束字符
start_loc = input_lst_s.find('[')
end_loc = input_lst_s.find(']')
rule, new_question = eval(input_lst_s[start_loc:end_loc+1])
# 接着使用拆分后的问题调用检索链
result = qa_chain.invoke(new_question)
result_context = result
# 接着调用输出格式解析
response = get_completion(prompt_output.format(new_question, result_context, rule))
response

'```python\n[\n    "生成式模型",\n    "判别式模型"\n]\n```'

可以看到，经过如上步骤，我们就成功地实现了输出格式的限定。当然，在上面代码中，核心为介绍 Agent 思想，事实上，不管是 Agent 机制还是 Parser 机制（也就是限定输出格式），LangChain 都提供了成熟的工具链供使用，欢迎感兴趣的读者深入探讨，此处就不展开讲解了。

通过上述讲解的思路，结合实际业务情况，我们可以不断发现 Bad Case 并针对性优化 Prompt，从而提升生成部分的性能。但是，上述优化的前提是检索部分能够检索到正确的答案片段，也就是检索的准确率和召回率尽可能高。那么，如何能够评估并优化检索部分的性能呢？下一章我们会深入探讨这个问题。